In [10]:
import pandas as pd
import psycopg2
import os

# List of CSV files and their corresponding table names
csv_files = [
    ('customers.csv', 'customers'),
    ('orders.csv', 'orders'),
    ('sellers.csv', 'sellers'),
    ('products.csv', 'products'),
    ('geolocation.csv', 'geolocation'),
    ('payments.csv', 'payments'),
    ('order_items.csv', 'order_items'),  # Added payments.csv for specific handling
]

# Connect to the PostgreSQL database
conn = psycopg2.connect(
    host='localhost',
    user='postgres',
    password='Achinty@',
    dbname='targetDB',
    port=5432
)
cursor = conn.cursor()

# Folder containing the CSV files
folder_path = r'C:\\Users\\Ujjwal\\Desktop\\SQL_Python_Ecommerce_Project\\Dataset'

def get_sql_type(dtype):
    if pd.api.types.is_integer_dtype(dtype):
        return 'INT'
    elif pd.api.types.is_float_dtype(dtype):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(dtype):
        return 'BOOLEAN'
    elif pd.api.types.is_datetime64_any_dtype(dtype):
        return 'TIMESTAMP'  # Use TIMESTAMP for PostgreSQL
    else:
        return 'TEXT'

for csv_file, table_name in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    
    # Debugging: Print the file path to check correctness
    print(f"Processing file: {file_path}")

    # Check if the file exists before trying to read it
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue
    
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    # Replace NaN with None to handle SQL NULL
    df = df.where(pd.notnull(df), None)
    
    # Debugging: Check for NaN values
    print(f"NaN values before replacement in {csv_file}:\n{df.isnull().sum()}\n")

    # Clean column names
    df.columns = [col.replace(' ', '_').replace('-', '_').replace('.', '_') for col in df.columns]

    # Generate the CREATE TABLE statement with appropriate data types
    columns = ', '.join([f'{col} {get_sql_type(df[col].dtype)}' for col in df.columns])
    create_table_query = f'CREATE TABLE IF NOT EXISTS {table_name} ({columns})'
    
    # Debugging: Print the create table query to check for issues
    print(f"Create Table Query for {table_name}:\n{create_table_query}\n")
    
    try:
        cursor.execute(create_table_query)
    except psycopg2.Error as e:
        print(f"Error executing create table query for {table_name}: {e}")
        continue

    # Insert DataFrame data into the PostgreSQL table
    for _, row in df.iterrows():
        # Convert row to tuple and handle NaN/None explicitly
        values = tuple(None if pd.isna(x) else x for x in row)
        sql = f"INSERT INTO {table_name} ({', '.join([col for col in df.columns])}) VALUES ({', '.join(['%s'] * len(row))})"
        
        try:
            cursor.execute(sql, values)
        except psycopg2.Error as e:
            print(f"Error inserting row into {table_name}: {e}")
            continue

    # Commit the transaction for the current CSV file
    conn.commit()

# Close the connection
conn.close()

Processing file: C:\\Users\\Ujjwal\\Desktop\\SQL_Python_Ecommerce_Project\\Dataset\customers.csv
NaN values before replacement in customers.csv:
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64

Create Table Query for customers:
CREATE TABLE IF NOT EXISTS customers (customer_id TEXT, customer_unique_id TEXT, customer_zip_code_prefix INT, customer_city TEXT, customer_state TEXT)

Processing file: C:\\Users\\Ujjwal\\Desktop\\SQL_Python_Ecommerce_Project\\Dataset\orders.csv
NaN values before replacement in orders.csv:
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

Create Table Query for orders:
CREATE TABLE IF 